In [1]:
import pandas as pd

In [2]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

# Coleta de Dados

In [3]:
# Dataframe com os dados das declarações (a seleção e tratamento dos dados foi feito no Receita Data)
df = pd.read_csv("Dados/declaracoes_2019.csv", sep=';', encoding = "latin1")

In [4]:
df.shape

(200000, 53)

In [5]:
# Dados com gini por estado (IBGE)
df_gini = pd.read_csv("Dados/gini_2019.csv", sep=';', encoding = "UTF-8")

In [6]:
df_gini.shape

(27, 3)

In [7]:
df_gini.head()

,estado,nome,gini
0,RO,Rondônia,0.471957
1,AC,Acre,0.556493
2,AM,Amazonas,0.568123
3,RR,Roraima,0.576438
4,PA,Pará,0.528684


In [8]:
# Dados de população por município
df_pop = pd.read_csv("Dados/populacao_2019.csv", sep=';', encoding = "UTF-8")

In [9]:
df_pop.shape

(5570, 5)

In [10]:
df_pop.head(17)

,UF,COD_UF,COD_MUNIC,NOME_MUNICIPIO,POPULACAO
0,RO,11,15,Alta Floresta D'Oeste,22945
1,RO,11,23,Ariquemes,107863
2,RO,11,31,Cabixi,5312
3,RO,11,49,Cacoal,85359
4,RO,11,56,Cerejeiras,16323
5,RO,11,64,Colorado do Oeste,15882
6,RO,11,72,Corumbiara,7391
7,RO,11,80,Costa Marques,18331
8,RO,11,98,Espigão D'Oeste,32374
9,RO,11,106,Guajará-Mirim,46174


In [11]:
# Tabela para relacionar o código de município utilizado na declaração e o código do IBGE 
df_mun = pd.read_csv("Dados/municipios_TOM.csv", sep=';', encoding = "latin1")

In [12]:
df_mun.shape

(5578, 6)

In [13]:
df_mun.head()

,nr_municipio,nb_municipio,latitude_municipio,longitude_municipio,cd_municipio_ibge,ae_municipio
0,-7.0,Inválido,-7.000,-7.000,IN,-7.000
1,-9.0,Não se aplica,-9.000,-9.000,NaN,-9.000
2,2125.0,Barrocas - BA,-11.529,-39.078,2903276,200.965
3,5564.0,Itanhangá - MT,-12.219,-56.638,5104542,2898.075
4,5565.0,Aroeiras do Itaim - PI,-7.077,-41.467,2200954,257.137


# Processamento / Tratamento dos Dados

## Declaração

### Excluir Coluna ID

In [14]:
# Excluir coluna id
df.drop(['id'],axis=1, inplace = True)

### Ocupação Principal

In [15]:
df.query("ocupacao=='NI' or ocupacao=='IN'").shape

(36454, 52)

In [16]:
# Transformar ocupação de string para numérico
df["ocupacao"] = pd.to_numeric(df["ocupacao"],errors='coerce' )

In [17]:
df.ocupacao.isna().sum()

36454

In [18]:
# Alterar valores nulos para 999 (Não Informado)
df["ocupacao"] = df["ocupacao"].fillna(999)
df.query("ocupacao==999").shape

(36454, 52)

In [19]:
df.ocupacao.isna().sum()

0

### Óbito

In [20]:
# Variável óbito possui o ano de óbito ou -9 caso contribuinte vivo
# Transformar a variável óbito em 0 ou 1. 
# 0 - Vivo     - se menor ou igual a 0
# 1 - Falecido - se maior que 0
df['obito'] = df.obito.apply(lambda x: 0 if x<=0 else 1)

In [21]:
df.obito.value_counts()

0    197803
1      2197
Name: obito, dtype: int64

In [22]:
df.shape

(200000, 52)

In [23]:
# Excluir Declarações com marca de óbito, pois podem não ter informações de todo o ano.
df.drop(df.loc[df['obito']==1].index, inplace=True)

In [24]:
df.shape

(197803, 52)

In [25]:
# Exclusão da variável óbito
df.drop(['obito'],axis=1, inplace = True)

### Criar Variável Alvo

In [26]:
def define_alvo(i):
    alvo = 1 # Considerado uma declaração normal
    # Alterar as que são anomalias
    # Malha fiscal finalizada com alteração de imposto devido
    if df.loc[i, 'auto'] == 1:
        alvo = -1
    # Declaração em malha fiscal
    elif df.loc[i, 'malha'] == 2:
        alvo = -1
    # Informou rendimentos abaixo da DIRF
    elif df.loc[i, 'dif_dirf'] > 0:
        alvo = -1
    # Declaração em malha preenchimento
    elif df.loc[i, 'malha'] == 1:
        alvo = -1
    # Informou dados de rendimentos com erro de preenchimento 
    elif df.loc[i, 'errop'] == 1:
        alvo = -1
    # Informou dados médicos com erro de preenchimento 
    elif df.loc[i,'errom'] == 1:
        alvo = -1
    return alvo

In [27]:
# Criação da variável alvo para identificação das anomalias
df['alvo'] = df.index.map(define_alvo)

In [28]:
df.alvo.value_counts()

 1    193093
-1      4710
Name: alvo, dtype: int64

In [29]:
# Excluir variáveis utilizadas para gerar a detecção das anomalias
df.drop(['malha', 'auto', 'errop', 'errom', 'dif_dirf'],axis=1, inplace = True)

### Corrigir alíquota
Corrigir valores de alíquota que estão com erro

In [30]:
df.aliquota.value_counts()

0.0     62829
7.5     41718
27.5    39050
15.0    22695
0.5     17603
22.5    13908
Name: aliquota, dtype: int64

In [31]:
def define_aliquota(num):
    if num < 22847.77:
        return 0
    elif num >= 22847.77 and num <33919.81:
        return 7.5
    elif num >= 33919.81 and num <45012.61:
        return 15
    elif num >= 45012.61 and num <55976.17:
        return 22.5
    elif num >= 55976.17:
        return 27.5       

In [32]:
df['aliquota'] = df.base_calculo.map(define_aliquota)

In [33]:
df.aliquota.value_counts()

0.0     68376
7.5     46137
27.5    43880
15.0    24404
22.5    15006
Name: aliquota, dtype: int64

### Excluir Exterior 
Os dados do IBGE são referentes ao Brasil, logo foi decidido retirar as declarações de residentes no exterior

In [34]:
df.query('estado=="EX"').estado.count()

183

In [35]:
df.query('estado!="EX"').estado.count()

197620

In [36]:
df.estado.count()

197803

In [37]:
df.drop(df.loc[df['estado']=='EX'].index, inplace=True)

In [38]:
df.estado.count()

197620

### Idade
Para melhorar os gráficos e análises, agrupar idades em que existem poucas declarações

In [39]:
# Agrupar acima de 100 anos
df.query('idade>90').idade.value_counts().sort_index()

91.0     150
92.0     123
93.0      89
94.0      64
95.0      50
96.0      40
97.0      19
98.0      21
99.0       9
100.0     11
101.0      5
102.0      7
103.0      1
104.0      2
105.0      1
107.0      6
108.0      1
111.0      1
112.0      2
113.0      1
114.0      1
116.0      1
117.0      1
Name: idade, dtype: int64

In [40]:
# Altera variável idade para 100 nos casos em que for maior que esse valor
df['idade'] = df.idade.apply(lambda x: 100 if x>100 else x)

In [41]:
# Agrupar abaixo de 15 anos
df.query('idade<18').idade.value_counts().sort_index(ascending=False)

17.0    96
16.0    60
15.0    49
14.0    40
13.0    36
12.0    29
11.0    21
10.0    18
9.0     19
8.0     26
7.0     14
6.0     13
5.0     13
4.0      9
3.0     10
2.0      4
1.0      4
Name: idade, dtype: int64

In [42]:
# Altera variável idade para 15 nos casos em que for menor que esse valor
df['idade'] = df.idade.apply(lambda x: 15 if x < 15 else x)

In [43]:
df.query('idade>100').idade.count()

0

In [44]:
df.query('idade<15').idade.count()

0

# Dados do IBGE

### GINI

In [45]:
df_gini.head()

,estado,nome,gini
0,RO,Rondônia,0.471957
1,AC,Acre,0.556493
2,AM,Amazonas,0.568123
3,RR,Roraima,0.576438
4,PA,Pará,0.528684


In [46]:
# Merge com dados de Gini por estado (left outer join)
df = pd.merge(df, df_gini[['estado', 'gini']], on='estado', how='left')

In [47]:
df[df.gini.isnull()].shape

(0, 48)

In [48]:
df.head()

,valor_dirf,rend_total,rend_tributavel,base_calculo,trabalho,aluguel,rend_capital,autonomo,livro_caixa,rural,ganho_capital,exterior,outros_rend,rend_isento,rend_exclusivo,contrib_prev,dependentes_ded,pensao_alimenticia,pensao_alimenticia_ded,geducativos,geducativos_ded,gmedicos,gmedicos_ded,aliquota,aliquota_efetiva,imposto_devido,imposto_devido2,resultado_decl,iap,iar,tipo_decl,completa,deducao,idade,sexo,casado,dependentes,alimentandos,estado,municipio,ocupacao,natureza,bens_ant,bens,dividas_ant,dividas,alvo,gini
0,0.00,25361.23,22762.00,18209.60,0.00,0.0,2599.23,22762.0,0.0,0.0,0.0,0.0,0.00,91.18,2508.05,1312.52,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.000000,0.00,0.00,0.00,0.00,0.0,3,0,4552.40,56.0,0,1,0.0,0.0,PR,4311.0,226.0,11,515877.95,529969.54,165365.12,148943.64,1,0.475764
1,46177.09,76082.23,46177.09,32271.11,74774.44,0.0,1307.79,0.0,0.0,0.0,0.0,0.0,0.00,26040.82,3864.32,0.00,0.0,0.0,0.0,0.0,0.0,13905.98,13905.98,7.5,0.009289,706.75,685.55,474.59,474.59,0.0,1,1,13905.98,82.0,1,0,0.0,0.0,TO,5432.0,999.0,61,251292.30,234255.05,0.00,0.00,1,0.529623
2,15.44,30927.00,28548.00,22838.40,30927.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,2379.00,2854.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.000000,0.00,0.00,0.00,0.00,0.0,1,0,5709.60,49.0,0,0,0.0,0.0,DF,5418.0,0.0,1,0.00,0.00,0.00,0.00,1,0.553359
3,0.00,21600.00,21600.00,17280.00,0.00,0.0,0.00,21600.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.000000,0.00,0.00,0.00,0.00,0.0,3,0,4320.00,34.0,1,1,0.0,0.0,ES,3401.0,214.0,11,69000.00,69000.00,0.00,0.00,1,0.518920
4,67121.50,92738.13,67121.50,53697.20,70974.12,0.0,0.00,0.0,0.0,0.0,0.0,0.0,21764.01,21764.01,3852.62,5675.95,0.0,0.0,0.0,0.0,0.0,4209.32,0.00,22.5,0.047967,4448.36,4448.36,-2016.80,0.00,2016.8,3,0,13424.30,56.0,0,1,0.0,0.0,RJ,2047.0,515.0,41,50000.00,50000.00,0.00,0.00,1,0.550819


### População

In [49]:
df_pop.head()

,UF,COD_UF,COD_MUNIC,NOME_MUNICIPIO,POPULACAO
0,RO,11,15,Alta Floresta D'Oeste,22945
1,RO,11,23,Ariquemes,107863
2,RO,11,31,Cabixi,5312
3,RO,11,49,Cacoal,85359
4,RO,11,56,Cerejeiras,16323


In [50]:
# Juntar os campos COF_UF e COD_MUNIC para formar o cd_municipio_ibge
df_pop['cd_municipio_ibge'] = df_pop['COD_UF'].map(str) + ("00000" + df_pop['COD_MUNIC'].map(str)).str[-5:]

In [51]:
df_pop.head()

,UF,COD_UF,COD_MUNIC,NOME_MUNICIPIO,POPULACAO,cd_municipio_ibge
0,RO,11,15,Alta Floresta D'Oeste,22945,1100015
1,RO,11,23,Ariquemes,107863,1100023
2,RO,11,31,Cabixi,5312,1100031
3,RO,11,49,Cacoal,85359,1100049
4,RO,11,56,Cerejeiras,16323,1100056


In [52]:
df_mun.head()

,nr_municipio,nb_municipio,latitude_municipio,longitude_municipio,cd_municipio_ibge,ae_municipio
0,-7.0,Inválido,-7.000,-7.000,IN,-7.000
1,-9.0,Não se aplica,-9.000,-9.000,NaN,-9.000
2,2125.0,Barrocas - BA,-11.529,-39.078,2903276,200.965
3,5564.0,Itanhangá - MT,-12.219,-56.638,5104542,2898.075
4,5565.0,Aroeiras do Itaim - PI,-7.077,-41.467,2200954,257.137


In [53]:
# Join entre a tabela de municípios da RFB e a tabela do IBGE (left outer join)
df_pop = pd.merge(df_pop, df_mun[['cd_municipio_ibge', 'nr_municipio']], 
                  on='cd_municipio_ibge', how='left')

In [54]:
df_pop.head()

,UF,COD_UF,COD_MUNIC,NOME_MUNICIPIO,POPULACAO,cd_municipio_ibge,nr_municipio
0,RO,11,15,Alta Floresta D'Oeste,22945,1100015,33.0
1,RO,11,23,Ariquemes,107863,1100023,7.0
2,RO,11,31,Cabixi,5312,1100031,37.0
3,RO,11,49,Cacoal,85359,1100049,9.0
4,RO,11,56,Cerejeiras,16323,1100056,27.0


In [55]:
df_pop[df_pop.nr_municipio.isnull()].shape

(0, 7)

In [56]:
# Inserir em df a população do município (left outer join)
df = pd.merge(df, df_pop[['nr_municipio', 'POPULACAO']], left_on='municipio', 
              right_on='nr_municipio', how='left')

In [57]:
df[df.POPULACAO.isnull()].shape

(0, 50)

In [58]:
df.drop(['municipio', 'nr_municipio', 'estado'], axis=1, inplace=True)

In [59]:
df.head()

,valor_dirf,rend_total,rend_tributavel,base_calculo,trabalho,aluguel,rend_capital,autonomo,livro_caixa,rural,ganho_capital,exterior,outros_rend,rend_isento,rend_exclusivo,contrib_prev,dependentes_ded,pensao_alimenticia,pensao_alimenticia_ded,geducativos,geducativos_ded,gmedicos,gmedicos_ded,aliquota,aliquota_efetiva,imposto_devido,imposto_devido2,resultado_decl,iap,iar,tipo_decl,completa,deducao,idade,sexo,casado,dependentes,alimentandos,ocupacao,natureza,bens_ant,bens,dividas_ant,dividas,alvo,gini,POPULACAO
0,0.00,25361.23,22762.00,18209.60,0.00,0.0,2599.23,22762.0,0.0,0.0,0.0,0.0,0.00,91.18,2508.05,1312.52,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.000000,0.00,0.00,0.00,0.00,0.0,3,0,4552.40,56.0,0,1,0.0,0.0,226.0,11,515877.95,529969.54,165365.12,148943.64,1,0.475764,19124
1,46177.09,76082.23,46177.09,32271.11,74774.44,0.0,1307.79,0.0,0.0,0.0,0.0,0.0,0.00,26040.82,3864.32,0.00,0.0,0.0,0.0,0.0,0.0,13905.98,13905.98,7.5,0.009289,706.75,685.55,474.59,474.59,0.0,1,1,13905.98,82.0,1,0,0.0,0.0,999.0,61,251292.30,234255.05,0.00,0.00,1,0.529623,299127
2,15.44,30927.00,28548.00,22838.40,30927.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,2379.00,2854.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.000000,0.00,0.00,0.00,0.00,0.0,1,0,5709.60,49.0,0,0,0.0,0.0,0.0,1,0.00,0.00,0.00,0.00,1,0.553359,3015268
3,0.00,21600.00,21600.00,17280.00,0.00,0.0,0.00,21600.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.000000,0.00,0.00,0.00,0.00,0.0,3,0,4320.00,34.0,1,1,0.0,0.0,214.0,11,69000.00,69000.00,0.00,0.00,1,0.518920,122499
4,67121.50,92738.13,67121.50,53697.20,70974.12,0.0,0.00,0.0,0.0,0.0,0.0,0.0,21764.01,21764.01,3852.62,5675.95,0.0,0.0,0.0,0.0,0.0,4209.32,0.00,22.5,0.047967,4448.36,4448.36,-2016.80,0.00,2016.8,3,0,13424.30,56.0,0,1,0.0,0.0,515.0,41,50000.00,50000.00,0.00,0.00,1,0.550819,150674


# Dataset Final

In [60]:
df.isna().sum()

valor_dirf                0
rend_total                0
rend_tributavel           0
base_calculo              0
trabalho                  0
aluguel                   0
rend_capital              0
autonomo                  0
livro_caixa               0
rural                     0
ganho_capital             0
exterior                  0
outros_rend               0
rend_isento               0
rend_exclusivo            0
contrib_prev              0
dependentes_ded           0
pensao_alimenticia        0
pensao_alimenticia_ded    0
geducativos               0
geducativos_ded           0
gmedicos                  0
gmedicos_ded              0
aliquota                  0
aliquota_efetiva          0
imposto_devido            0
imposto_devido2           0
resultado_decl            0
iap                       0
iar                       0
tipo_decl                 0
completa                  0
deducao                   0
idade                     0
sexo                      0
casado              

In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 197620 entries, 0 to 197619
Data columns (total 47 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   valor_dirf              197620 non-null  float64
 1   rend_total              197620 non-null  float64
 2   rend_tributavel         197620 non-null  float64
 3   base_calculo            197620 non-null  float64
 4   trabalho                197620 non-null  float64
 5   aluguel                 197620 non-null  float64
 6   rend_capital            197620 non-null  float64
 7   autonomo                197620 non-null  float64
 8   livro_caixa             197620 non-null  float64
 9   rural                   197620 non-null  float64
 10  ganho_capital           197620 non-null  float64
 11  exterior                197620 non-null  float64
 12  outros_rend             197620 non-null  float64
 13  rend_isento             197620 non-null  float64
 14  rend_exclusivo      